In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h2o

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-spotify-hit-predictor-dataset/LICENSE
/kaggle/input/the-spotify-hit-predictor-dataset/dataset-of-70s.csv
/kaggle/input/the-spotify-hit-predictor-dataset/dataset-of-10s.csv
/kaggle/input/the-spotify-hit-predictor-dataset/README.txt
/kaggle/input/the-spotify-hit-predictor-dataset/dataset-of-60s.csv
/kaggle/input/the-spotify-hit-predictor-dataset/dataset-of-80s.csv
/kaggle/input/the-spotify-hit-predictor-dataset/dataset-of-00s.csv
/kaggle/input/the-spotify-hit-predictor-dataset/dataset-of-90s.csv


In [4]:
from h2o.automl import H2OAutoML
h2o.init(max_mem_size="15G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5ambgn5_
  JVM stdout: /tmp/tmp5ambgn5_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5ambgn5_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,2 months and 28 days
H2O_cluster_name:,H2O_from_python_unknownUser_3rfaxy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [20]:
data=pd.read_csv("../input/the-spotify-hit-predictor-dataset/dataset-of-00s.csv")
data
data.drop("uri",axis=1)
train=data[0:5000]
test=data[5000:]

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
5000,Kids on Holiday,Animal Collective,spotify:track:2ud2CsFhbMRjuzSZTSEsNE,0.491,0.6160,8,-11.967,1,0.0415,0.4590,0.000292,0.1440,0.515,88.456,346413,4,21.00714,17,0
5001,Make It Wit Chu,Queens of the Stone Age,spotify:track:0Cgd3CK6D2hhbwmLRvtscH,0.704,0.8110,9,-5.308,1,0.0277,0.3550,0.000189,0.1300,0.578,91.970,290493,4,70.36922,7,0
5002,Chasing Cars,Snow Patrol,spotify:track:11bD1JtSjlIgKgZG2134DZ,0.563,0.5920,9,-4.571,1,0.0276,0.2090,0.000051,0.1320,0.136,104.016,267933,4,19.37993,17,1
5003,Death by Blonde,DeVotchKa,spotify:track:3h3NppKUFOMfUP3j89qSXj,0.411,0.8030,4,-7.259,1,0.0457,0.6810,0.002170,0.1190,0.829,162.702,214901,4,77.33996,6,0
5004,X Gon' Give It To Ya,DMX,spotify:track:1zzxoZVylsna2BQB65Ppcb,0.761,0.8990,10,-3.090,0,0.1830,0.0135,0.000000,0.0719,0.673,95.027,217587,4,86.71927,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,Summer Rain,Carl Thomas,spotify:track:0NBHHa8wwwmBnn3aAzX5wJ,0.667,0.6270,6,-10.488,0,0.0654,0.0972,0.000052,0.1110,0.784,186.081,232560,4,40.87045,10,1
5868,And I,Ciara,spotify:track:1Jp9n1uHB72CfK31j4mEPh,0.691,0.3890,6,-10.125,1,0.0653,0.2550,0.000000,0.0981,0.437,122.219,233840,4,81.77735,7,1
5869,"Mass in B minor BWV 232, Missa: Duetto - Chris...",Johann Sebastian Bach,spotify:track:4NIOi1ImMfdufRTsgoKjbD,0.297,0.0773,2,-23.839,1,0.0620,0.9510,0.000217,0.1210,0.401,75.916,275560,4,37.51903,11,0
5870,Loog,The Clean,spotify:track:2Qyj2nUdm8y37TCCzDasFn,0.390,0.6010,7,-8.236,0,0.0291,0.0313,0.947000,0.1190,0.439,116.122,223627,4,39.84092,11,0


In [30]:
y="Hit on the basis of chorus"
frame=h2o.H2OFrame(train)
x=h2o.H2OFrame(train[2:5000])

aml=H2OAutoML(max_models=50,seed=50,max_runtime_secs=600)
aml.train(y="chorus_hit",training_frame=frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [38]:
leader=aml.leaderboard
leader
lead=aml.leader
lead
test_frame=h2o.H2OFrame(test)
preds=aml.predict(test_frame)
preds

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
36.9539
65.0582
32.8596
59.7817
64.3336
32.2843
37.9263
36.2046
58.1535
39.3154
